Reference# Adversarial Attacks with FGSM by PyImageSearch
https://www.pyimagesearch.com/2021/03/01/adversarial-attacks-with-fgsm-fast-gradient-sign-method/

In [2]:


from __future__ import print_function
import numpy as np
import os
import tensorflow as tf

from tensorflow import keras
#import tensorflow.python.keras as keras
from tensorflow.keras.layers import Dense, Conv2D, BatchNormalization, Activation
from tensorflow.keras.layers import AveragePooling2D, Input, Flatten, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, LearningRateScheduler
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.regularizers import l2
from tensorflow.keras import backend as K
from tensorflow.keras.models import Model, load_model
import numpy as np
from numpy import loadtxt
from tensorflow.keras.utils import plot_model, to_categorical
from tensorflow.keras.losses import MSE
from sklearn import metrics
import pandas as pd


In [10]:

def mse(pred, target):
  return tf.square(pred-target)/2

def generate_image_adversary(model, image, label, eps=2 / 255.0):
    # cast the image
    image = tf.cast(image, tf.float32)
    # record our gradients
    print('value of epsilon: ', eps)
    with tf.GradientTape() as tape:
        tape.watch(image) # explicitly indicate that our image should be tacked for gradient updates
        pred = model(image) ## use our model to make predictions on the input image and
        #print('shape of pred:', pred.shape, 'shape of label:', label.shape)
        #print('pred: ', pred)
        loss = MSE(label, pred) # compute the loss
        print('loss', loss)

    # calculate the gradients of loss with respect to the image, then
    # and compute the sign of the gradient
    gradient = tape.gradient(loss, image)
    signedGrad = tf.sign(gradient)
    #print('sign', signedGrad)
    # construct the image adversary
    adversary = (image + (signedGrad * eps)).numpy()

    # return the image adversary to the calling function
    return adversary

In [7]:
########### CONFIG ######################
# Load and Normalize data.
currentNetworkName = 'original_model_5250_5250'
datasetName = 'ad1_43_UR_0_5250_5250'
dataDir = 'assets/'
dataFileName = dataDir + datasetName + '.sample'
meta_data = 0
cols = 22 + (meta_data * 2) # <2 uid rid> <8 user-attribs> <2 user_meta> <8 res-attribs> <2 res-meta> <4 ops>
########### CONFIG END ###################

# load the dataset
raw_dataset = loadtxt(dataFileName, delimiter=' ', dtype=np.str)
dataset = raw_dataset[:,2:cols] # TO SKIP UID RID
#np.random.shuffle(dataset)

# split into user-resource pair and operations variables
feature = dataset.shape[1]
print('Features:', feature)
attribs = feature - 4

urp = dataset[:,0:attribs]
operations = dataset[:,attribs:feature]

uattrib_end = 8
rattrib_end = 16
umeta_end = uattrib_end + meta_data
rmeta_end = rattrib_end + meta_data

urp = np.delete(urp, slice(uattrib_end, umeta_end), 1)
print(urp.shape)
urp = np.delete(urp, slice(rattrib_end, rmeta_end), 1)
print(urp.shape)
feature = urp.shape[1]
print('Features after meta data removal:', feature)
print(urp[0])
print(operations[0])
#split into training and test data
eval_size = (int)(urp.shape[0] * 0.8) #20% of total dataset

print('evaluation data size: ' + str(eval_size)) 

############### ENCODING ##############
urp = to_categorical(urp)
print('shape of URP after encoding')
print(urp.shape)
#######################################

# test data
urp_test = urp[:eval_size,0:feature]
operations_test = operations[:eval_size,0:4]
print('urp_test shape:', urp_test.shape)

# trainin data
urp_train = urp[eval_size:,0:feature]
operations_train = operations[eval_size:,0:4]
print('urp_train shape:', urp_train.shape)

#determine batch size
batch_size = min(urp_train.shape[0]/10, 16)
print('batch size: ' + str(batch_size))

x_train = urp_train[..., np.newaxis]
x_test = urp_test[..., np.newaxis]
print('shape of URT-Train after adding new dimension')
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)

y_train = operations_train
y_test = operations_test

print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')
print('y_train shape:', y_train.shape)

# Run training, without data augmentation.

DIR_ASSETS = 'assets/'
PATH_MODEL = DIR_ASSETS + currentNetworkName + '.hdf5'

if os.path.exists(PATH_MODEL):
    print('Loading trained model from {}.'.format(PATH_MODEL))
    current_nw = load_model(PATH_MODEL)
else:
    print('No trained model found at {}.'.format(PATH_MODEL))
    exit(0)

print ('****************** ======== ******************')
y_test_current_nw = current_nw.predict(x_test)
y_test_current_nw = (y_test_current_nw > 0.5).astype(int)

#print('successful!')

# grab a sample and its label
image = x_test[0]
label = y_test[0].astype(float)

# generate a sample adversary for the current sample and 
# make the prediction on the adversary
print('Shape of image: ', image.shape)
actual_pred = current_nw.predict(image.reshape(1, 16, 138, 1))
actual_pred = (actual_pred > 0.5).astype(int)
print('Actual Prediction', actual_pred)

adversary = generate_image_adversary(current_nw, image.reshape(1, 16, 138, 1), label, eps=0.2)
#print('adversarial image')
#print(adversary.reshape(16, 138))
advers_pred = current_nw.predict(adversary)
advers_pred = (advers_pred > 0.5).astype(int)
print('predict adversarial image', advers_pred)


Features: 20
(43, 16)
(43, 16)
Features after meta data removal: 16
['9' '23' '58' '45' '44' '48' '6' '18' '9' '82' '13' '46' '44' '38' '6'
 '45']
['1' '0' '1' '0']
evaluation data size: 34
shape of URP after encoding
(43, 16, 138)
urp_test shape: (34, 16, 138)
urp_train shape: (9, 16, 138)
batch size: 0.9
shape of URT-Train after adding new dimension
x_train shape: (9, 16, 138, 1)
x_test shape: (34, 16, 138, 1)
9 train samples
34 test samples
y_train shape: (9, 4)
Loading trained model from assets/original_model_5250_5250.hdf5.
****************** ======== ******************
Shape of image:  (16, 138, 1)
Actual Prediction [[1 0 0 0]]
value of epsilon:  0.2
loss tf.Tensor([[1.5511192e-08 2.4499921e-06 4.9993467e-01 6.0245549e-07]], shape=(1, 4), dtype=float32)
predict adversarial image [[1 1 0 0]]


In [9]:
from PIL import Image

'''
img_adv = adversary.reshape(16, 138)
img = image.reshape(16, 138)
print(img.shape)
non_zero_count = 0
for i in range(img.shape[0]):
  for j in range(img.shape[1]):
    if(img[i][j] != 0):
      non_zero_count = non_zero_count + 1
      print(img_adv[i][j]) # to see the pixel's value change in adversarial image 
print('total non zero pixels', non_zero_count)
'''

'''

print(img_adv.shape)
non_zero_count = 0
for i in range(img_adv.shape[0]):
  for j in range(img_adv.shape[1]):
    if(img_adv[i][j] != 0):
      non_zero_count = non_zero_count + 1
print('total non zero pixels', non_zero_count)
'''
output = Image.fromarray(image.reshape(16, 138), 'RGB')
output.save('input_image.png')

output = Image.fromarray(adversary.reshape(16, 138), 'RGB')
output.save('adversary_image.png')
